In [1]:
#!pip install openpyxl

In [2]:
#Import
import pandas as pd
import numpy as np
import scipy
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
#Display two point cloud with different colours in one o3d window
def display_two_pointclouds(array_1,array_2,windowname):
    p1_pcd = o3d.geometry.PointCloud()
    p1_pcd.points = o3d.utility.Vector3dVector(array_1)
    p1_pcd.paint_uniform_color([1, 0.706, 0])


    p2_pcd = o3d.geometry.PointCloud()
    p2_pcd.points = o3d.utility.Vector3dVector(array_2)
    p2_pcd.paint_uniform_color([0, 0.706, 1])

    concate_pc = np.concatenate((array_1, array_2),axis = 0)
    p1_color = np.asarray(p1_pcd.colors)
    p2_color = np.asarray(p2_pcd.colors)
    p3_color = np.concatenate((p1_color,p2_color), axis=0)

    p3_pcd = o3d.geometry.PointCloud()
    p3_pcd.points = o3d.utility.Vector3dVector(concate_pc)
    p3_pcd.colors = o3d.utility.Vector3dVector(p3_color)
    o3d.visualization.draw_geometries([p3_pcd],window_name = windowname)

The purpose of this function is to take a vessel and determine at which axial distance at which another vessel branches off. Additionally it needs to check to see what the terminus condition of the vessel is and then create the appropriate ending conditions for it whether that be flowing directly into another vessel (continous), branching into multiple vessels (split) or no end condition in which case we will need to apply the 0D end condition (tree).

Our first necessary step will be to import the excel file which contains all of the file names and the links between them

In [4]:
#Import Excel Sheets
try:
    arteries_sheet = pd.read_excel('C:\\Users\\Cassidy.Northway\\GitRemoteRepo\\FlowTracker.xlsx', sheet_name = 0)
    veins_sheet = pd.read_excel('C:\\Users\\Cassidy.Northway\\GitRemoteRepo\\FlowTracker.xlsx', sheet_name = 1)
except:
    arteries_sheet = pd.read_excel('C:\\Users\\cbnor\\Documents\\Full Body Flow Model Project\\FlowTracker.xlsx', sheet_name = 0)
    veins_sheet = pd.read_excel('C:\\Users\\cbnor\\Documents\\Full Body Flow Model Project\\FlowTracker.xlsx', sheet_name = 1)
    

In [5]:
#Select sheet
sheet = arteries_sheet
#sheet = veins_sheet

In [6]:
#Define our data frame
df = pd.DataFrame(columns=['Name','Centre Axis Array', 'Radius Array','End Condition'])


In [8]:
#Now we need to define our 
location = 0

#Look at every vessel in the sheet
for index in range(551,577):#range(0,sheet.shape[0]):

    name = sheet.at[index,'Anatomy Name']
    file_name = sheet.at[index,'Filename']
    
    #Determine whether the vessel branches at all
    end_point = sheet.at[index,'End Point']

    if pd.isna(end_point):
        final_condition = 'LW' #indicating that the end condition will be the Lax Wendroff
    else:
        #If the branches exist then we need to process the 
        final_condition = end_point.split(',')
        final_condition = [s.strip() for s in final_condition]
    
    #Does the vessel branch (other than the end condition?)
    branches = sheet.at[index,'Out Flow']
    
    if pd.isna(branches):
        segement_tag  = False
    else:
        segement_tag = True
        branches = branches.split(',')
        branches = [s.strip() for s in branches]
        
        #Remove end condition branches 
        if final_condition != 'LW':
            for vessels in final_condition:
                branches.remove(vessels)

    #If there are no segements then we just save the whole thing
    if segement_tag == False:
        main_branch_filename = file_name + '_fitted_data.npy'
    
        try:
            main_branch_array = np.load('C:\\Users\\Cassidy.Northway\\GitRemoteRepo\\FittedVesselsFiles\\' + main_branch_filename)
        except:
            main_branch_array = np.load('C:\\Users\\cbnor\\Documents\\Full Body Flow Model Project\\FittedVesselsFiles\\' + main_branch_filename)
        sub_name =  name + '_0'
        
        center_sub_array = main_branch_array[:,0:3 ]
        radius_array = main_branch_array[:,3 ]
        end_condition = final_condition
        new_row_seg = {'Name' : sub_name,'Centre Axis Array': center_sub_array, 'Radius Array' : radius_array,'End Condition' : end_condition }
        df.loc[len(df)] = new_row_seg
                
    #If the vessel does segement now we gotta set spicy and determine where it segements for each vessel and which comes first
    if segement_tag == True:
        seg_df = pd.DataFrame(columns=['Branch Name','Index of Split'])
        sub_index = 0
        
        #Find the file names of the main and branches
        main_branch_filename = file_name + '_fitted_data.npy'
        branch_filenames = []

        #For each of the branching files determine the file name to load in the gile
        for i in range(0,len(branches)):
            branch_name = branches[i]
            vessel_row = sheet[sheet['Anatomy Name'].str.match(branch_name)].index.values[0]
            branch_filename = sheet.at[sheet.index[vessel_row],'Filename']
            branch_filename = branch_filename + '_fitted_data.npy'
            branch_filenames.append(branch_filename)

        #Now we import all of the vessel file
        try:
            main_branch_array = np.load('C:\\Users\\Cassidy.Northway\\GitRemoteRepo\\FittedVesselsFiles\\' + main_branch_filename)
        except:
            main_branch_array = np.load('C:\\Users\\cbnor\\Documents\\Full Body Flow Model Project\\FittedVesselsFiles\\' + main_branch_filename)

        for i in range(0,len(branches)):
            branch_filename = branch_filenames[i]
            branch_name = branches[i]
        
            try:
                branch_array = np.load('C:\\Users\\Cassidy.Northway\\GitRemoteRepo\\FittedVesselsFiles\\' + branch_filename)
            except:
                branch_array = np.load('C:\\Users\\cbnor\\Documents\\Full Body Flow Model Project\\FittedVesselsFiles\\' + branch_filename)


            #Find the nearest points
            dist_array = scipy.spatial.distance.cdist(main_branch_array[:,0:3],branch_array[:,0:3])
            dist_array = dist_array[:,0]

            index_split = np.where (np.min(dist_array) == dist_array)[0]
            seg_df.loc[len(seg_df)] = {'Branch Name': branch_name , 'Index of Split': index_split[0]}
        
        #We now have the number of off branching vessels and where they branch so now we need to now save the 
        #segements and off branches and sort segment frame by distance along vessel
        
        seg_df = seg_df.sort_values(by ='Index of Split')
        seg_df = seg_df.reset_index(drop=True)
        intial_index = 0

        for i in range(0,len(seg_df)+1):
            if i != len(seg_df):
                sub_name =  name + '_' + str(i)
                final_index = seg_df.at [ i , 'Index of Split']
                center_sub_array = main_branch_array[intial_index:final_index+1,0:3 ]
                radius_array = main_branch_array[intial_index:final_index,3 ]
                end_condition = [name + '_' + str(i+1), branches[i]+'_0' ]
                new_row_seg = {'Name' : sub_name,'Centre Axis Array': center_sub_array, 'Radius Array' : radius_array,'End Condition' : end_condition }
                df.loc[len(df)] = new_row_seg
                intial_index = final_index
            else:
                sub_name =  name + '_' + str(i)
                final_index = -1
                center_sub_array = main_branch_array[intial_index:final_index,0:3 ]
                radius_array = main_branch_array[intial_index:final_index,3 ]
                if final_condition != 'LW':
                    end_condition = [final_condition[0] +'_0']
                else:
                    end_condition = final_condition
                new_row_seg = {'Name' : sub_name,'Centre Axis Array': center_sub_array, 'Radius Array' : radius_array,'End Condition' : end_condition }
                df.loc[len(df)] = new_row_seg
             

## Reassess end conditions

We want to go through rows and change the end condition from the name of the vessel segement to the index of those vessels

In [33]:
#Go row by row of the data frame

for i in range (0,len(df)):
    end_condition = df.at[i,'End Condition']
    if end_condition != 'LW':
        replacement = []
        if isinstance(end_condition[0], str): 
            for j in range(0,len(end_condition)):
                condition = end_condition[j]

                index = df[df['Name']==condition].index.values[0]
                replacement.append(index)
            df.at[i,'End Condition'] = replacement 
            


                 Name                                  Centre Axis Array  \
0    arteries_rleg0_0  [[-107.4632424731635, -5.938743137985156, -133...   
1    arteries_rleg1_0  [[-114.96594771700069, -4.516197961396698, -14...   
2    arteries_rleg1_1  [[-114.88989852866645, -2.403195732162193, -14...   
3    arteries_rleg1_2  [[-114.88989852866645, -2.403195732162193, -14...   
4    arteries_rleg1_3  [[-114.69956053652277, -0.2642367914375668, -1...   
..                ...                                                ...   
65  arteries_rleg28_0  [[-197.49080264872237, -117.06490042604094, -1...   
66  arteries_rleg29_0  [[-214.6341143795293, -69.02840543378994, -103...   
67  arteries_rleg29_1  [[-213.9998099410211, -76.97964714852628, -104...   
68  arteries_rleg29_2  [[-213.87402560440543, -93.67073608487095, -10...   
69  arteries_rleg29_3  [[-213.87402560440543, -93.67073608487095, -10...   

                                         Radius Array End Condition  
0   [1.0006416883

In [34]:
df.to_pickle('rleg.pkl')